In [ ]:
import sys
sys.path.insert(0, '../code')
import numpy as np
import os 
import matplotlib.pyplot as plt
from inversion import invert
from conj_grad import norm
from params import t_d,x_d,y_d,t0,x0,y0,Nt

In [ ]:
data = np.load('../data/h_obs.npy')

eps_w = 1e-3      # regularization parameter

In [ ]:
sol,fwd,mis = invert(data,eps_w)

In [ ]:
# import cycler
# from matplotlib.colors import rgb2hex
# color = plt.cm.Accent(np.linspace(0, 1,Nt))

In [ ]:
if os.path.isdir('inv_pngs')==False:
    os.mkdir('inv_pngs')

levels_data = np.arange(-5,6,1)
levels_inv = np.arange(0,26,2.)

# basal vertical velocity threshold for estimating lake boundary 
Q = np.abs(sol)
Q_max = np.max(Q)

Q_mean = np.mean(Q,axis=(1,2))
Q_std = np.std(Q,axis=(1,2))

thresh = 2

i0 = 0    
for i in range(Nt): 
    plt.close()
    fig = plt.figure(figsize=(16,14))
    plt.suptitle(r'$t=$ '+'{:.2f}'.format(t_d[i])+' d',fontsize=30)
    plt.subplot(221)
    plt.title("ICESat-2 data (ATL15)",fontsize=24)
    plt.contourf(x_d/1e3,y_d/1e3,data[i,:,:],levels=levels_data,cmap='coolwarm',extend='both')
    plt.ylabel(r'$y$ (km)',fontsize=20)
    plt.gca().xaxis.set_ticklabels([])
    plt.yticks(fontsize=16)
   
    plt.subplot(222)
    plt.title("Modelled data",fontsize=24)
    p = plt.contourf(x_d/1e3,y_d/1e3,fwd[i,:,:],levels=levels_data,cmap='coolwarm',extend='both')
    plt.gca().xaxis.set_ticklabels([])
    plt.gca().yaxis.set_ticklabels([])
 
    
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.875, 0.53, 0.02, 0.35])
    cbar = fig.colorbar(p,cax=cbar_ax,orientation='vertical',ticks=levels_data)
    cbar.ax.get_yaxis().labelpad = 10
    cbar.ax.tick_params(labelsize=18)  
    cbar.set_label(r'$\Delta h$ (m)',fontsize=24)
    cbar.ax.tick_params(labelsize=16)
    
    plt.subplot(223)
    plt.title("Lake activity \"imprint\" ",fontsize=24)

    ind = np.arange(i0,i,1)
    for j in ind:
        alpha_j = 0.01 + 0.99*(float(j-i0+1)/float(i-i0+1))
        color_j = 'k'#rgb2hex(color[j])
        if np.max(Q[j,:,:])>thresh:
            plt.contour(x_d/1e3,y_d/1e3,Q[j,:,:],levels=[thresh],linewidths=1,linestyles=['-'],colors=color_j,alpha=alpha_j)
    if np.max(Q[i,:,:])>thresh:   
        plt.contour(x_d/1e3,y_d/1e3,Q[i,:,:],levels=[thresh],linewidths=3,linestyles=['-'],colors='k')
    else:
        i0 = i
        plt.contour(x_d/1e3,y_d/1e3,Q[0,:,:],levels=[thresh],linewidths=3,linestyles=['-'],colors='w')
        
    
    plt.xlabel(r'$x$ (km)',fontsize=20)
    plt.ylabel(r'$y$ (km)',fontsize=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    
    plt.subplot(224)
    plt.title("Inversion",fontsize=24)
    p = plt.contourf(x_d/1e3,y_d/1e3,Q[i,:,:],levels=levels_inv,cmap='Blues',extend='both')
    
    if np.max(Q[i,:,:])>thresh:
        c = plt.contour(x_d/1e3,y_d/1e3,Q[i,:,:],levels=[thresh],linewidths=3,linestyles=['-'],colors='k')
  
    
    plt.xlabel(r'$x$ (km)',fontsize=20)
  
    plt.xticks(fontsize=16)
    plt.gca().yaxis.set_ticklabels([])

    
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.875, 0.11, 0.02, 0.35])
    cbar = fig.colorbar(p,cax=cbar_ax,orientation='vertical',ticks=levels_inv)
    cbar.ax.get_yaxis().labelpad = 10
    cbar.ax.tick_params(labelsize=18)  
    cbar.set_label(r'$|w_b|$ (m/yr)',fontsize=24)
    cbar.ax.tick_params(labelsize=16) 
    if np.max(Q[i,:,:])>thresh:
        cbar.add_lines(c)
        cbar.lines[0].set_linestyles(c.linestyles)
    
   # plt.tight_layout()
    plt.savefig('inv_pngs/inv_'+str(i),bbox_inches='tight')
    plt.close()

to make a movie: open terminal, cd to inv_pngs, and use the ffmpeg command

`ffmpeg -r 5 -f image2 -s 1920x1080 -i inv_%01d.png -vcodec libx264 -pix_fmt yuv420p -vf scale=1280:-2 movie.mp4`